In [1]:
import pickle
import requests
import tenseal as ts
from cc_he_utils import json_serialize, json_deserialize, load_test_data_np, load_test_data_df, setup_tenseal_context
from ppxgboost import PPBooster as ppbooster
from ppxgboost.PPBooster import MetaData
from ppxgboost import PaillierAPI as paillier

In [2]:
xtest_np, ytest_np = load_test_data_np()
xtest_df, ytest_df = load_test_data_df()

In [3]:
context = setup_tenseal_context()

def make_nn_query():
    enc_input = ts.ckks_vector(context, xtest_np[0])
    request_data = {
        'input': json_serialize(enc_input.serialize()),
        'context': json_serialize(context.serialize()),
    }
    response = requests.post('http://134.226.86.101:8000/nn', json=request_data, timeout=10).json()
    enc_x = ts.ckks_vector_from(context, json_deserialize(response['result']))
    return enc_x.decrypt()

In [14]:
prf_key = pickle.load(open('prf_key.pt', "rb"))
encrypter = pickle.load(open('encrypter.pt', "rb"))
min_max = pickle.load(open('min_max.pt', "rb"))
private_key = pickle.load(open('private_key.pt', "rb"))

def make_xgboost_query():
    enc_input = xtest_df.head(1).copy()
    ppbooster.enc_input_vector(prf_key, encrypter, set(xtest_df.columns), enc_input, MetaData(min_max))
    request_data = {
        'input': json_serialize(enc_input)
    }
    response = requests.post('http://134.226.86.101:8000/xgboost', json=request_data, timeout=10).json()
    encrypted_result = json_deserialize(response['result'])[0]
    return paillier.decrypt(private_key, encrypted_result)

In [10]:
%time make_nn_query()

CPU times: user 1.94 s, sys: 492 ms, total: 2.43 s
Wall time: 4.02 s


[-0.3354605077784836]

In [15]:
%time make_xgboost_query()

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 206 ms


-5.397705921856